# Lesson 7: Saving and Loading Models
: 저장해놓으면 새로운 거 추가해서 우리가 끝난데서 더 training해서 넣고 하면 편할텐데..의 연장선상이다.
*  **Tensorflow**는 이런 점에서 굉장히 편하다.
  -  저장한 모델을 바로 mobile 등의 platform에 deploy할 수 있게 'Saved model format'을 지원, direct로 연결할 수 있는 기능 지원.
  - **'Saved model'** : weight, models architecture, optimizer configuration 등 다양한 info들 제공.
*   **Train, Save, Load it back** 이 이번 keywords.

# Part 1: Load the Cats vs. Dogs Dataset

In [ ]:
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.



*  **with_info** : True로 설정해서 dataset으로 부터 metadata를 불러올 수 있게하는 함수
*   **as_supervised** : True로 설정해서 image에 corresponding하는 label을 얻을 수 있음
*   **split=splits** : 우리가 정의한대로 80 train, 20 validation data split이 가능하도록 하는 것


In [ ]:
def format_image(image, label):
  # `hub` image modules exepct their data normalized to the [0,1] range.
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return  image, label

num_examples = info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches      = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)


*   map 을 batch 앞에 썼다. 여기는 지금 size가 다 다른데, **Tf에서는 다른 size의 tensor들을 batch할 수 없기** 때문.


# Part 2: Transfer Learning with TensorFlow Hub

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

feature vector을 pre-trained MobileNet model이 있는 Tf hub로 부터 다운받는다.

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


two output이 나오게 하는, new classification head로 새로운 Keras sequential model을 만들어 준다.

## Train the model

We now train this model like any other, by first calling `compile` followed by `fit`.
**Calling `compile` 그리고 optimizer specifying**.

# Part 3: Save as Keras `.h5` model

we can save it as an **HDF5 file**, which is the **format used by Keras**. Our HDF5 file will have the extension '.h5'

In [ ]:
t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)


*   t 줄을 추가함으로써, timestamp 대로 저장이 되는데, time에 따라 **다른 이름으로** 저장된다.
*   **model.save** 라는 save method를 쓴다. **path 지정만** 하면 된다.
*   **'!ls'** 커맨드를 치면 있는 문서 목록이 쭉 뜬다. 체킹 용도인듯.

# Part 4:  Load the Keras `.h5` Model

We will now load the model we just saved into a new model called `reloaded`. We will need to provide the file path and the `custom_objects` parameter. This **parameter tells keras** how to load the `hub.KerasLayer`** from the `feature_extractor` **we used for transfer learning. 원래는 path만 지정해도 된다.

In [ ]:
reloaded = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

In [ ]:
(abs(result_batch - reloaded_result_batch)).max()


*  우리 모델을 'reloaded' 라는 variable로 로드하는 거다. : origin model과 **구별되게** 해준다. 
*   이 시점에서 체킹하는 건 아직 아무것도 안건드린 똑같은 모델이기 때문에, 같은 image_batch 로 predict했을 때 같은 result가 나오는 지 보는 것이다.
여기서 new data를 얻어서 이 data로 모델을 굴리고 싶다면 "Keep training" 파트로 가면 된다.
*   **'abs'** 는 절댓값 도출함수다.result 값 차이를 보려고 하는 건데, 여기서는 0.0 즉, 완벽히 일치하는 값을 도출해줬다.


# Keep Training

Besides making predictions, we can also take our `reloaded` model and keep training it. To do this, you can just train the `reloaded` as usual(똑같이 하면 된다. 다만.), using the **`.fit` method.**

In [ ]:
EPOCHS = 3
history = reloaded.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

# Part 5: Export as SavedModel

In [ ]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(model, export_path_sm)

**`tf.saved_model.save()` function**. This functions takes in the model we want to save and the path to the folder where we want to save our model. 

This function will create a folder where you will find an `assets` folder, a `variables` folder, and the `saved_model.pb` file. 

# Part 7: Loading the SavedModel as a Keras Model

The object returned by `tf.saved_model.load` is **not a Keras object** (i.e. doesn't have `.fit`, `.predict`, `.summary`, etc. methods). Therefore, you can't simply take your `reloaded_sm` model and keep training it by running `.fit`. To be able to get back a full keras model from the Tensorflow SavedModel format **we must use **the `tf.keras.models.load_model` function. This function will work the same as before, except now we pass the path to the folder containing our SavedModel. 이것도 결국 path만 지정해주면 끝날일이다.

-다만 Part 7의 코드 중 윗부분이 API를 불러오는 과정에서 bug가 있는데, 후에는 밑만 필요하고 위는 안필요해질듯.